<a href="https://colab.research.google.com/github/shingrajia18/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df.dtypes)


Attrition     object
Department    object
dtype: object


In [4]:
# Create a list of at least 10 column names to use as X data
columns = ["Education", "Age", "DistanceFromHome", "JobSatisfaction", "OverTime", "StockOptionLevel", "WorkLifeBalance", "YearsAtCompany", "YearsSinceLastPromotion", "NumCompaniesWorked"]



# Create X_df using your selected columns
X_df = attrition_df[columns]

X_df = pd.get_dummies(X_df, columns=["OverTime"], dtype=int)

# Show the data types for X_df
print(X_df.dtypes)


Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
OverTime_No                int64
OverTime_Yes               int64
dtype: object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [6]:
# Convert your X data to numeric data types however you see fit
X_train = X_train.astype('int64')
X_test = X_test.astype('int64')

# Add new code cells as necessary
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1102, 11) (368, 11) (1102, 2) (368, 2)


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [8]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc_Department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
enc_Department.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
y_train_encoded_Department = enc_Department.transform(y_train['Department'].values.reshape(-1, 1))
y_test_encoded_Department = enc_Department.transform(y_test['Department'].values.reshape(-1, 1))


y_test_encoded_Department[:5]



array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
enc_Attrition = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
enc_Attrition.fit(y_train['Attrition'].values.reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
y_train_encoded_Attrition = enc_Attrition.transform(y_train['Attrition'].values.reshape(-1, 1))
y_test_encoded_Attrition = enc_Attrition.transform(y_test['Attrition'].values.reshape(-1, 1))

y_test_encoded_Attrition[:5]


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_features = X_train_scaled.shape[1]

# Create the input layer
inputs = layers.Input(shape=(input_features))

# Create at least two shared layers
shared_1 = layers.Dense(64, activation='relu', name='shared1')(inputs)
shared_2 = layers.Dense(128, activation='relu', name='shared2')(shared_1)

In [11]:
# Create a branch for Department with a hidden layer and an output layer

# Create the hidden layer
hidden_1 = layers.Dense(32, activation='relu', name='department_hidden')(shared_2)


# Create the output layer
output_1 = layers.Dense(3, activation='softmax', name='department_output')(hidden_1)


In [12]:
# Create a branch for Attrition with a hidden layer and an output layer

# Create the hidden layer
hidden_2 = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_2)

# Create the output layer
output_2 = layers.Dense(2, activation='sigmoid', name='attrition_output')(hidden_2)


In [13]:
# Create the model
model = Model(inputs=inputs, outputs=[output_1, output_2])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 11)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   768       ['input_1[0][0]']             
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [14]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_encoded_Department, 'attrition_output': y_train_encoded_Attrition}, epochs=100, shuffle=True, verbose=2)


Epoch 1/100
35/35 - 1s - loss: 1.4208 - department_output_loss: 0.8519 - attrition_output_loss: 0.5689 - department_output_accuracy: 0.6062 - attrition_output_accuracy: 0.7477 - 1s/epoch - 36ms/step
Epoch 2/100
35/35 - 0s - loss: 1.2016 - department_output_loss: 0.7597 - attrition_output_loss: 0.4419 - department_output_accuracy: 0.6661 - attrition_output_accuracy: 0.8276 - 50ms/epoch - 1ms/step
Epoch 3/100
35/35 - 0s - loss: 1.1651 - department_output_loss: 0.7495 - attrition_output_loss: 0.4156 - department_output_accuracy: 0.6624 - attrition_output_accuracy: 0.8303 - 49ms/epoch - 1ms/step
Epoch 4/100
35/35 - 0s - loss: 1.1365 - department_output_loss: 0.7395 - attrition_output_loss: 0.3970 - department_output_accuracy: 0.6679 - attrition_output_accuracy: 0.8421 - 47ms/epoch - 1ms/step
Epoch 5/100
35/35 - 0s - loss: 1.1204 - department_output_loss: 0.7344 - attrition_output_loss: 0.3860 - department_output_accuracy: 0.6706 - attrition_output_accuracy: 0.8466 - 65ms/epoch - 2ms/step
E

In [15]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {'department_output': y_test_encoded_Department, 'attrition_output': y_test_encoded_Attrition}, verbose=2)
print(results)

12/12 - 0s - loss: 4.4648 - department_output_loss: 3.3416 - attrition_output_loss: 1.1233 - department_output_accuracy: 0.4918 - attrition_output_accuracy: 0.8505 - 245ms/epoch - 20ms/step
[4.464828014373779, 3.3415699005126953, 1.1232587099075317, 0.49184781312942505, 0.8505434989929199]


In [16]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {results[3]}")
print(f"Attrition Accuracy: {results[4]}")

Department Accuracy: 0.49184781312942505
Attrition Accuracy: 0.8505434989929199


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I think accuracy can be used in this model to evaluate the model's performance. This is a good result, but there is still room for improvement. The model can be further optimized by tuning the hyperparameters, increasing the number of epochs, or adding more layers to the neural network. Additionally, more data can be collected to improve the model's performance. Overall, the model is a good starting point for HR to predict employee attrition and department fit.
2. I chose relu for shared layer because it is a good activation function for hidden layers in neural networks. I chose sigmoid for the attrition output layer because it is a good activation function for binary classification problems. It squashes the output to a range between 0 and 1, which is useful for predicting probabilities. I chose softmax for the department output layer because it is a good activation function for multi-class classification problems. It squashes the output to a range between 0 and 1, which is useful for predicting probabilities. Additionally, it ensures that the sum of the output values is equal to 1, which is useful for interpreting the results.
3. A model might be improved by increasing the number of epochs, adding more layers to the neural network, or tuning the hyperparameters. Additionally, more data can be collected to improve the model's performance. Overall, the model is a good starting point for HR to predict employee attrition and department fit.